<a href="https://colab.research.google.com/github/kuheli31/Fake-News-Detector/blob/main/Fake_News_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import re
import string
import requests
from requests.utils import quote

**G-News API (*Real News*)**

In [ ]:
#Real time news api (https://gnews.io/dashboard) -- 100 requests per day
news_api_key = 'd2b27a1e3f818564545508b6c744d24a'
#Endpoints are -> {search , top-headlines}
endpoint = 'search'
#Parameters are -> {The topic the user want to search}
question = input('Enter your question: ')
parameters = f'q={quote(question)}&lang=en&max=5'
#Example (try this link to understand the url)-> https://gnews.io/api/v4/top-headlines?q=modi&lang=en&max=5&apikey=d2b27a1e3f818564545508b6c744d24a
news_api_url = f'https://gnews.io/api/v4/{endpoint}?{parameters}&apikey={news_api_key}'

response = requests.get(news_api_url)
data = response.json()

if 'articles' in data:
    for article in data['articles']:
        print(f"Published Time: {article['publishedAt']}")
        print(f"Title: {article['title']}")
        print(f"Description: {article['description']}")
        print(f"Image: {article['image']}")
        print(f"Content: {article['content']}")
        print(f"Source Name: {article['source']['name']}")
        print(f"Source URL: {article['source']['url']}")
        print(f"Source Country: {article['source']['country']}")
        print(f"URL: {article['url']}\n")
else:
    print(data)

Enter your question: g20 meet
Published Time: 2025-08-21T12:06:08Z
Title: India-Russia To Strengthen Trade Ties To Offset Trump Tariffs, BIG Takeaways From Jaishankar-Lavrov Meet
Description: EAM S. Jaishankar and Russian Foreign Minister Sergey Lavrov reaffirmed India–Russia’s deep-rooted ties in Moscow, pledging cooperation in labour, trade, and connectivity. The leaders announced new consulates, backed anti-terror efforts, and vowed closer engagement in BRICS, G20, and SCO.
Image: https://img.republicworld.com/all_images/jaishankar-lavrov-meet-1755781005544-16_9.webp
Content: External Affairs Minister S. Jaishankar and his Russian counterpart Sergey Lavrov on Thursday underlined that India–Russia ties remain anchored in “trust” and “shared interests”, while opening the door to new avenues of cooperation. The leaders addre... [1929 chars]
Source Name: Republic World
Source URL: https://www.republicworld.com
Source Country: in
URL: https://www.republicworld.com/business/india-russia-t

**Twitter API**

In [ ]:
# Importing necessary libraries for X (Twitter) API interaction
import tweepy

# Hardcoded Bearer Token
X_API_BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAKCJ4QEAAAAAcEQy2tIw03BHpMFUg3fg9sbpGu8%3Db2Q2D7tbJUI2A0fJjOOMz69iKILmCmXeBYwVNNoxOaty3y2Oj2"

#Searching in tweets
question = input('Enter your search query for X (Twitter): ').strip()

# --- Main Execution Block ---
if not X_API_BEARER_TOKEN or not question:
    print("ERROR: Both the Bearer Token and the search query are required.")
else:
    try:
        # Initialize Tweepy Client
        client = tweepy.Client(X_API_BEARER_TOKEN)

        # API Call with image support
        response = client.search_recent_tweets(
            query=f'{question} -is:retweet has:images lang:en',
            max_results=10,
            tweet_fields=['created_at', 'public_metrics', 'attachments'],
            user_fields=['name', 'username', 'verified'],
            media_fields=['url', 'preview_image_url', 'type'],
            expansions=['author_id', 'attachments.media_keys']
        )

        if not response or not response.data:
            print("\nNo recent tweets found for that query.")
        else:
            tweets = response.data
            users = {user['id']: user for user in response.includes.get('users', [])} if response.includes else {}
            media = {m['media_key']: m for m in response.includes.get('media', [])} if response.includes else {}

            print("\n--- Found Tweets ---\n")
            for tweet in tweets:
                author = users.get(tweet.author_id)
                media_urls = []

                # Extract media URLs
                if hasattr(tweet, 'attachments') and 'media_keys' in tweet.attachments:
                    for key in tweet.attachments['media_keys']:
                        if key in media and media[key].type == 'photo':
                            media_urls.append(media[key].url)

                tweet_url = f"https://x.com/{author.username}/status/{tweet.id}" if author else None
                author_url = f"https://x.com/{author.username}" if author else None

                print(f"Published Time: {tweet.created_at}")
                print(f"Tweet Text: {tweet.text}")
                if author:
                    print(f"Author Name: {author.name} (@{author.username})")
                    print(f"Author URL: {author_url}")
                print(f"Tweet URL: {tweet_url}")

                if media_urls:
                    print("Image URLs:")
                    for img in media_urls:
                        print(f"- {img}")
                else:
                    print("No images found.")

                print("\n")

    except tweepy.errors.Unauthorized:
        print("\n--- AUTHENTICATION ERROR ---")
        print("The Bearer Token you provided is invalid. Please check your credentials on the X Developer Portal.")
    except tweepy.errors.BadRequest:
        print("\n--- INVALID SEARCH QUERY ---")
        print("Your search query is invalid or too complex. Please check the format.")
    except Exception as e:
        print(f"\nAn unexpected error occurred.")
        print(f"Error details: {e}")


Enter your search query for X (Twitter): moon

An unexpected error occurred.
Error details: 429 Too Many Requests
Too Many Requests


**Reddit API**

In [ ]:
!pip install praw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 5.0 MB/s eta 0:00:00


In [ ]:
# Importing necessary libraries for Reddit API interaction
import praw
import datetime # To format the timestamp from Reddit

# --- API Configuration ---
REDDIT_CLIENT_ID = 'utnSmiRRrv9ZDIlJ9LveMg'
REDDIT_CLIENT_SECRET = 'jzP9XRolmRsu3CNiDZmKF0IDDGcXpA'
REDDIT_USER_AGENT = 'python:moonsearcher:v1.0 (by /u/Dependent_Car_5879)'

# --- Search Parameters ---
try:
    question = input('Enter your search query for Reddit: ').strip()
    limit_str = input('How many results would you like? (e.g., 10): ').strip()
    num_results = int(limit_str) if limit_str.isdigit() else 10
except EOFError:
    print("\nNo input provided for search query. Exiting.")
    exit()

# --- Main Execution Block ---
if not REDDIT_CLIENT_ID or not REDDIT_CLIENT_SECRET or not question:
    print("ERROR: Client ID, Client Secret, and a search query are required.")
else:
    try:
        reddit = praw.Reddit(
            client_id=REDDIT_CLIENT_ID,
            client_secret=REDDIT_CLIENT_SECRET,
            user_agent=REDDIT_USER_AGENT,
        )

        search_results = reddit.subreddit("all").search(
            query=question,
            sort='new',
            limit=num_results
        )

        submissions = list(search_results)

        if not submissions:
            print("\nNo recent posts found for that query.")
        else:
            print(f"\n--- Found {len(submissions)} Posts ---\n")
            for submission in submissions:
                author_name = submission.author.name if submission.author else "[deleted]"
                post_url = f"https://www.reddit.com{submission.permalink}"
                author_url = f"https://www.reddit.com/user/{author_name}" if author_name != "[deleted]" else "N/A"

                published_time = datetime.datetime.fromtimestamp(submission.created_utc)

                # --- Print basic info ---
                print(f"Published Time: {published_time.strftime('%Y-%m-%d %H:%M:%S')}")
                print(f"Post Title: {submission.title}")
                print(f"Subreddit: r/{submission.subreddit.display_name}")
                print(f"Score: {submission.score} (Upvote Ratio: {submission.upvote_ratio * 100:.0f}%)")
                print(f"Author: u/{author_name}")
                print(f"Author URL: {author_url}")
                print(f"Post URL: {post_url}")

                # --- Print full text if available ---
                if submission.selftext:
                    print(f"Post Text: {submission.selftext.replace(chr(10), ' ')}")

                # --- Extract image if any ---
                image_url = None
                if submission.url.endswith(('.jpg', '.jpeg', '.png', '.gif')):
                    image_url = submission.url
                elif hasattr(submission, 'preview'):
                    try:
                        image_url = submission.preview['images'][0]['source']['url']
                    except (KeyError, IndexError):
                        pass

                if image_url:
                    print(f"Image URL: {image_url}")
                else:
                    print("Image: None")

                print("\n" + "-"*60 + "\n")

    except praw.exceptions.PRAWException as e:
        print("\n--- API ERROR ---")
        print("An error occurred with the Reddit API. This could be due to invalid credentials or a connection problem.")
        print(f"Error details: {e}")
    except Exception as e:
        print(f"\nAn unexpected error occurred.")
        print(f"Error details: {e}")


Enter your search query for Reddit: roads india
How many results would you like? (e.g., 10): 10


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.




--- Found 10 Posts ---

Published Time: 2025-10-16 06:35:51
Post Title: Misled by Triumph showroom about price hike — lost ₹20-25k before Diwali offers
Subreddit: r/triumph4India
Score: 1 (Upvote Ratio: 100%)
Author: u/cipherthread
Author URL: https://www.reddit.com/user/cipherthread
Post URL: https://www.reddit.com/r/triumph4India/comments/1o7yzfm/misled_by_triumph_showroom_about_price_hike_lost/
Post Text:  I went to the Triumph showroom on 10th September after hearing that prices would increase after 22nd September due to a GST revision. The showroom staff also confirmed this and advised me to book the bike before 22nd to avoid the price hike.  So I paid 100k on 14th September and they completed the RTO registration and insurance by 20th September later i found out that the prices actually didn't increase after 22nd September and in fact Triumph launched Diwali offers in October.  I had clearly told them that I wanted delivery on 18th October (Dhanteras), but they still pushed me t

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Image: None

------------------------------------------------------------

Published Time: 2025-10-16 04:14:42
Post Title: Some people deserve to be poor
Subreddit: r/pune
Score: 10 (Upvote Ratio: 66%)
Author: u/ayxsh_03
Author URL: https://www.reddit.com/user/ayxsh_03
Post URL: https://www.reddit.com/r/pune/comments/1o7wmjw/some_people_deserve_to_be_poor/
Post Text: Sometimes I genuinely feel that some people deserve to be poor because of the way they behave. In India, a shocking number of people lack even the most basic civic sense, and the rest of us end up paying the price for their ignorance and arrogance. Just yesterday, I faced three separate incidents in traffic that made me lose faith in people's sense of responsibility. Even after giving the correct left indicator, a scooty guy behind me kept honking aggressively, glaring at me, and even threatening to beat me up. It's unbelievable how people can be so impatient and uncivilized. Later, at a green signal that clearly allowed u

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Image: None

------------------------------------------------------------

Published Time: 2025-10-16 03:56:23
Post Title: Fix your country
Subreddit: r/IndianTeenagers
Score: 1 (Upvote Ratio: 100%)
Author: u/No-Match-6725
Author URL: https://www.reddit.com/user/No-Match-6725
Post URL: https://www.reddit.com/r/IndianTeenagers/comments/1o7wafr/fix_your_country/
Post Text: I just wanna say maturing is realising how diverse and big india actually is , and leave the motherland is the retarded way of solving shit ,   I have heard of some garbage cafe where you get food in exchange of plastic waste and many other thing , and I have seen individuals and organisations working to clean or country by all means of fixing , maybe it's the minorities of people that runs   idk why I am Posting this here but it just feels right , be aware, save your trash and put it in the right, not on the side of the road . If you want high living standards make it yourself, it for everyone and throw trash and make

**Data Preprocessing**

In [11]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 10.3 MB/s eta 0:00:00


In [12]:
!pip install beautifulsoup4 spacy scikit-learn


In [13]:
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 104.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [10]:
import re
import json
from bs4 import BeautifulSoup
import emoji
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import AgglomerativeClustering
from datetime import datetime
from urllib.parse import urlparse

# Load English model for NLP (NER, sentence segmentation)
nlp = spacy.load("en_core_web_sm")  # or "en_core_web_trf" for transformer-based embeddings

# --- Helper functions ---

def clean_text(text):
    """Clean text: remove HTML, emojis, normalize whitespace, lowercase."""
    if not text:
        return ""
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()
    # Remove emojis
    text = emoji.replace_emoji(text, replace='')
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text)
    # Lowercase
    text = text.lower().strip()
    return text

def extract_urls(text):
    """Extract URLs from text"""
    return re.findall(r'https?://\S+', text)

def extract_hashtags(text):
    """Extract hashtags"""
    return re.findall(r'#\w+', text)

def normalize_timestamp(ts):
    """Convert timestamp string to UTC datetime"""
    try:
        return datetime.fromisoformat(ts.replace('Z', '+00:00'))
    except:
        return None

def extract_claims(text):
    """Extract potential claims from text using sentence segmentation + NER"""
    doc = nlp(text)
    claims = []
    for sent in doc.sents:
        # If sentence contains at least one named entity, consider as claim candidate
        if len(sent.ents) > 0:
            claims.append(sent.text.strip())
    return claims

def cluster_claims(claims, n_clusters=5):
    """Optional: cluster similar claims using TF-IDF + AgglomerativeClustering"""
    if not claims:
        return []
    vectorizer = TfidfVectorizer(stop_words='english')
    X = vectorizer.fit_transform(claims)
    n_clusters = min(n_clusters, len(claims))
    clustering = AgglomerativeClustering(n_clusters=n_clusters).fit(X.toarray())
    clustered_claims = {}
    for idx, label in enumerate(clustering.labels_):
        clustered_claims.setdefault(label, []).append(claims[idx])
    return clustered_claims

# --- Example processing function ---

def process_post(post_json):
    """
    Input: raw JSON of a tweet or Reddit post
    Output: normalized dict with cleaned text, claims, URLs, hashtags, user info
    """
    # Text
    text = post_json.get('text') or post_json.get('selftext') or ""
    clean = clean_text(text)

    # URLs & hashtags
    urls = extract_urls(clean)
    hashtags = extract_hashtags(clean)

    # Timestamp
    timestamp = normalize_timestamp(post_json.get('created_utc') or post_json.get('created_at'))

    # User info (public only)
    user_info = post_json.get('user') or post_json.get('author')
    user_data = {}
    if user_info:
        user_data = {
            'username': user_info.get('screen_name') or user_info.get('name') or user_info.get('author'),
            'followers_count': user_info.get('followers_count', 0),
            'following_count': user_info.get('friends_count', 0),
            'account_age_days': (datetime.utcnow() - normalize_timestamp(user_info.get('created_at'))).days
            if user_info.get('created_at') else None,
            'verified': user_info.get('verified', False)
        }

    # Claims extraction
    claims = extract_claims(clean)

    return {
        'clean_text': clean,
        'urls': urls,
        'hashtags': hashtags,
        'timestamp': timestamp,
        'user': user_data,
        'claims': claims
    }

# --- Example usage ---
if __name__ == "__main__":
    # Example post JSON
    sample_post = {
        "text": "India-Russia trade ties are strengthening! Read more: https://example.com/news #G20 #Trade",
        "created_at": "2025-10-16T03:49:04Z",
        "user": {
            "screen_name": "Klutzy_Agency_1992",
            "followers_count": 1200,
            "friends_count": 300,
            "created_at": "2018-06-01T12:00:00Z",
            "verified": False
        }
    }

    processed = process_post(sample_post)
    print(json.dumps(processed, indent=4))

    # Optional: cluster claims
    clustered = cluster_claims(processed['claims'], n_clusters=3)
    print("\nClustered Claims:")
    print(json.dumps(clustered, indent=4))


ModuleNotFoundError: No module named 'emoji'